# Global Terrorism Attacks - Predicting The Responsible Group

**Table of Contents**
<div id="toc"></div>

## Load data

In [1]:
%matplotlib inline

import pandas as pd
import csv
from tqdm import tqdm_notebook
from sklearn import preprocessing

gtd = pd.read_csv('gtd_converted.csv', encoding='latin1', low_memory=False)
gtd.columns
gtd.tail(5)

,Unnamed: 0,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
170345,170345,201612310040,2016,12,31,NaN,0,NaN,146,Niger,...,NaN,"""15 soldiers killed, 19 injured in Niger terro...","""Niger: Boko Haram attack kills two soldiers i...",NaN,START Primary Collection,1,1,0,1,NaN
170346,170346,201612310042,2016,12,31,NaN,0,NaN,160,Philippines,...,NaN,"""AFP braces for more BIFF attacks in ARMM,"" Ma...","""Asia: US Army ASD Philippine Terrorist Media ...",NaN,START Primary Collection,-9,-9,0,-9,NaN
170347,170347,201612310043,2016,12,31,NaN,1,NaN,229,Democratic Republic of the Congo,...,NaN,"""DRC Armed Forces Dislodge Ugandan LRA Rebels ...",NaN,NaN,START Primary Collection,1,1,0,1,NaN
170348,170348,201612310044,2016,12,31,NaN,0,NaN,130,Mexico,...,There is doubt that this incident meets terror...,"""Gunmen shoot at home of Mexican indigenous ri...","""â Gunmen shoot at home of Mexican indigenou...",NaN,START Primary Collection,-9,-9,0,-9,NaN
170349,170349,201701270001,2016,12,30,NaN,0,NaN,195,Sudan,...,There is doubt that this incident meets terror...,"""Rebels To Blame For Nierteti Killing: Central...","""Security deteriorates in West Darfur alongsid...",NaN,START Primary Collection,0,0,0,0,NaN


In [2]:
# # Mapping for _txt columns
# txt_maps = {col[:-4]:{} for col in gtd.columns if col[-4:]=='_txt'}
# for index, row in tqdm_notebook(gtd.iterrows(), desc='Rows'):
#     for key in txt_maps:
#         if pd.notnull(row[key]):
#             txt_maps[key][int(row[key])] = row[key+'_txt']
#TODO: Fix progress bar - tqdm
#  # Save maps to CSV
# with open('mapping.csv', 'w') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(txt_maps.items())

## Preprocessing

### Filter Columns

In [3]:
# keep_cols = ['gname', 'iyear', 'imonth', 'iday', 'country', 'provstate', 'city', 'region', 'crit1', 'crit2', 'crit3', 'doubtterr', 'success', 'attacktype1', 'targtype1', 'targsubtype1', 'natlty1', 'weaptype1', 'weapsubtype1', 'nperps', 'nkill', 'nwound', 'ransom']
keep_cols = ['gname', 'iyear', 'country', 'crit1', 'crit2', 'crit3', 'attacktype1', 'targtype1', 'targsubtype1', 'weaptype1', 'weapsubtype1', 'ransom']
# keep_cols = ['gname', 'iyear', 'country', 'attacktype1', 'weaptype1']

gtd = gtd[keep_cols]
gtd.tail(5)

,gname,iyear,country,crit1,crit2,crit3,attacktype1,targtype1,targsubtype1,weaptype1,weapsubtype1,ransom
170345,Boko Haram,2016,146,1,1,0,9,4,34.0,13,NaN,NaN
170346,Unknown,2016,160,1,1,1,3,12,NaN,6,16.0,NaN
170347,Lord's Resistance Army (LRA),2016,229,1,1,1,6,14,75.0,13,NaN,0.0
170348,Unknown,2016,130,0,1,1,1,10,53.0,5,3.0,NaN
170349,Sudan Liberation Movement,2016,195,1,1,1,2,14,75.0,5,5.0,NaN


### Filter Groups

In [4]:
from collections import Counter

group_count = Counter(gtd['gname'])
# Remove groups with 3 attacks or less
filtered_groups = [group for group, counter in group_count.items() if counter > 3]
# Remove 'Unknown'
filtered_groups.remove('Unknown')

gtd = gtd[gtd['gname'].isin(filtered_groups)]

#TODO: Change floats to bools

In [5]:
### Transform Target

In [6]:
number = preprocessing.LabelEncoder()
gtd['gname'] = number.fit_transform(gtd.gname) #number.inverse_transform()
gtd = gtd.rename(columns={'gname': 'groupId'})

### Fill NaNs

In [7]:
gtd=gtd.fillna(0) #TODO: 999 or 0?

gtd.tail(5)

,groupId,iyear,country,crit1,crit2,crit3,attacktype1,targtype1,targsubtype1,weaptype1,weapsubtype1,ransom
170342,939,2016,4,1,1,0,9,4,27.0,13,0.0,0.0
170343,939,2016,4,1,1,1,3,17,94.0,6,16.0,0.0
170345,214,2016,146,1,1,0,9,4,34.0,13,0.0,0.0
170347,555,2016,229,1,1,1,6,14,75.0,13,0.0,0.0
170349,919,2016,195,1,1,1,2,14,75.0,5,5.0,0.0


In [8]:
gtd.to_csv('gtd_processed_11features.csv', encoding='utf-8', index=False)